# Basic QPQ algorithm


- 1: Estimate the preference $θ_i(r)$
- 2: Calculate the normalized preference $\bar{\theta}_i(r)=\mathit{PIT}_i(\theta_i(r))$
- 3: Declare a value $\dot{\theta}_i(r)$ that represents her normalized preference.
- 4: Wait to receive the published normalized preferences  ̇$\dot{\theta}_j(r)$ from all players
- 5: For all $j \in N$ do
    - 6: if not GoF_Test($\dot{\theta}_j(r)$) then
        - 7:  ̈$\ddot{\theta}_j(r) \leftarrow \hat{\theta}_j(r)$, where $\hat{\theta}_j(r) := \textrm{Pseudorandom}(\dot{\theta}_{-j}(r))$)
    - 8: else
        - 9:  ̈$\ddot{\theta}_j(r)$ $\leftarrow$ $\dot{\theta}_{j}(r)$
    - 10: end if
    - 11: $\mathit{History}_j$ $\leftarrow$ $\mathit{History}_j \cup \{\ddot{\theta}_j(r)\}$
- 12: end for
- 13: Let $d=\arg \max_{j\in\mathcal{N}} \{ \ddot{\theta}_j(r) \}$
- 14: if $d = i$ then
    - 15: Player i gets resource r
- 16: end if

In [ ]:
# Values is a matrix (N x r) or (number of playes x number of rounds)
def QPQ(declaredVals, trueVals, histlen, maxpval, KSTest=True, debug=True):
    dims = declaredVals.shape
    N = dims[0] # number of players
    R = dims[1] # number of rounds
    decisions = np.full(R, np.nan)
    historic = np.full((N, histlen), np.nan) # Empty history matrix (N x histlen) or (number of playes x History Len)
    
    utilities = np.zeros((N, R)) # Empty utility matrix (N x R)
    falsenegatives = np.zeros((N, R)) # Empty utility matrix (N x R)
    for i in range(R):
        # Roll historic to the left
        if (i > histlen):
            historic = np.roll(historic, -1, axis=1)
            
        theta = np.zeros(N)
        # copy declared values at the end of the historic
        historic[:, min(i, histlen - 1)] = declaredVals[: , i]
        
        for j in range(N):
            if debug: 
                print ("player ", j, " has values ", historic[j])
            if KSTest and stats.kstest(historic[j, 0:min(i+1, histlen)], 'uniform').pvalue < (1 - maxpval):
                if debug: print("False negative")
                theta[j] = np.random.uniform(0, 1, 1)
                falsenegatives[j, i] = 1
            else:
                theta[j] = declaredVals[j, i]
                
        # copy declared values at the end of the historic
        historic[:, min(i, histlen - 1)] = theta
        
        d = int(np.argmin(theta))
        if debug: print("Win player ", d)
        decisions[i] = d
        utilities[d, i] = trueVals[d, i]
    return decisions, utilities, falsenegatives